In [1]:
import numpy as np

from coppertop.pipe import *
from coppertop.dm.core.types import offset
from coppertop.dm.core import kvs, first, values, drop, at, keys, shape, count, collect, pad, take
from coppertop.dm.numpy import shape
from coppertop.dm.core.types import pylist

from coppertop.dm.examples.cluedo.core import *
from coppertop.dm.examples.cluedo import simple
from coppertop.dm.examples.cluedo import bayes_pad
from coppertop.dm.examples.cluedo.bayes_pad import PP
from coppertop.dm.examples.cluedo.games import games

In [2]:
deal, preevents, events = games[13]
# deal = {Pl: [St, Ki, Ha, Ca, Or], Gr: 5, Or: 4, Pe: 4}

In [3]:
Me, hand = deal >> kvs >> first
handSizes = [len(hand)] + list(deal >> values >> drop >> 1)
knowns = [hand] + [[]] * (len(deal) - 1)

f'{deal >> keys | pylist}' >> PP;

[Pl, Gr, Or, Pe]


In [4]:
_.possMaster, slices = bayes_pad.createPossibilities(people, weapons, rooms, handSizes, knowns)

nTBI:  150  nEachSet:  90090   nCols:  16  mem:  216.216MB
zeros: 0.0007980000000000001  =0:  0.040130625


In [5]:
_.pads = {}
_.events = []
_.suggestId = 0
_.otherPlayers = deal >> keys >> drop >> 1
_.rowTitles = ([TBI] + _.otherPlayers) >> collect >> (lambda x: repr(x) >> pad(_,{'left':5}))
_.knownCards = dict(zip(deal >> keys, knowns))
_.ss = dict(zip([TBI, Me] + _.otherPlayers, slices))
_.DEBUG = True

s0, s1, s2, s3 = 1.05, 0.5, 0.25, 0.0

_.poss = np.array(_.possMaster)
_.l = np.ones(_.poss >> shape >> at >> (0 | offset))
f'{(_.l >> count) * 8 / 1_000_000:,.1f} MB' >> PP;

108.1 MB


In [6]:
# events = []

In [104]:
preevents = []
like = {0:100, 1:10, 2:5, 3:0}
bag = simple.createBag(Me, cards, {Mu: 4, Gr: 4, Or: 3, Pe: 3}) \
    >> figureKnown >> preevents + events \
    >> processResponses >> events2 \
    >> processSuggestions(_, _, like) >> events2
bag >> PP;

We know Orchid has {Ki}
Orchid: 7 -> 2
We know Mustard has {Di}
Peacock: 21 -> 11
We know Peacock has {Ha}
We know Green has {Li}
We know Orchid has {Bi}
We know Orchid has {Bi}
3300000
6400000
1010000
55500000
                                  Me          Mu          Gr          Or          Pe          
Green            -         131%         -          ? 63%       ? 48%        ?          ? 19%  
Mustard          X                      -           -           -           -           -     
Orchid           -      S1 125%         -           -          ? 6%        ? 99%       ? 19%  
Peacock          -                      X           -           -           -           -     
Plum             -      S1 77%          -          ? 9%        ? 48%        ?          ? 19%  
Scarlet          -                      X           -           -           -           -     
----                                                                                          
Candlestick      -           

In [7]:
events >> bayes_pad.processEvents(_, s0, s1, s2, s3)

calcPad: #0, 6,056.3ms
0: 13513500,  1: 0,  2: 0,  3: 0
unchanged: 13513500, 216.2 MB
9355500
old: 13513500, 216.2 MB
new: 4158000, 66.5 MB
calcPad: #1, 2,501.0ms
0: 2236500,  1: 1921500,  2: 226800,  3: 0
unchanged: 4158000, 66.5 MB
1921500
old: 4158000, 66.5 MB
new: 2236500, 35.8 MB
update: <coppertop.dm.examples.cluedo.core.HasOne object at 0x11d970f50> has one of [Pe, Le, Ki]   (already has [])
919800
old: 2236500, 35.8 MB
new: 1316700, 21.1 MB
calcPad: #2, 495.6ms
0: 916020,  1: 400680,  2: 0,  3: 0
unchanged: 1316700, 21.1 MB
283500
old: 1316700, 21.1 MB
new: 1033200, 16.5 MB
0
unchanged: 1033200, 16.5 MB
calcPad: #3, 407.9ms
0: 810180,  1: 223020,  2: 0,  3: 0
unchanged: 1033200, 16.5 MB
0
unchanged: 1033200, 16.5 MB
calcPad: #4, 434.0ms
0: 1033200,  1: 0,  2: 0,  3: 0
unchanged: 1033200, 16.5 MB
681240
old: 1033200, 16.5 MB
new: 351960, 5.6 MB
274260
old: 351960, 5.6 MB
new: 77700, 1.2 MB
27300
old: 77700, 1.2 MB
new: 50400, 0.8 MB
calcPad: #5, 20.4ms
0: 0,  1: 50400,  2: 15960

In [50]:
-3 >> bayes_pad.printBayesPad
'' >> PP
-2 >> bayes_pad.printBayesPad
'' >> PP
-1 >> bayes_pad.printBayesPad

        Gr     Mu     Or     Pe     Pl     Sc     Ca     Da     Le     Re     Ro     Wr     Ba     Bi     Co     Di     Ha     Ki     Li     Lo     St     
TBI      -      -      -      -      -   100.0%    -      -      -      -      -   100.0%    -    12.9%  52.7%  12.9%    -      -     8.6%  12.9%    -  
Gr     36.0%  36.0%    -    65.5%  36.0%    -      -   100.0%    -    68.2%  10.8%    -      -    31.1%    -    31.1%    -      -    54.5%  31.1%    -  
Or     44.9%  44.9%    -      -    44.9%    -      -      -      -    29.1%   6.6%    -   100.0%  38.6%    -    38.6%    -      -    13.9%  38.6%    -  
Pe     19.2%  19.2%    -    34.5%  19.2%    -      -      -   100.0%   2.7%  82.6%    -      -    17.4%  47.3%  17.4%    -      -    23.0%  17.4%    -  

        Gr     Mu     Or     Pe     Pl     Sc     Ca     Da     Le     Re     Ro     Wr     Ba     Bi     Co     Di     Ha     Ki     Li     Lo     St     
TBI      -      -      -      -      -   100.0%    -      -      -      -  